# Interacting with Campaigns <a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Create campaigns](#create)
1. [Interact with campaigns](#interact)
1. [Batch recommendations](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have several solutions and at least one solution version for each. Once a solution version is created, it is possible to get recommendations from them, and to get a feel for their overall behavior.

This notebook starts off by deploying each of the solution versions from the previous notebook into individual campaigns. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

As you with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [7]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import pandas as pd

In [8]:
%store -r

In [9]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Test Campaigns

Now that our campaigns have been fully created, let's test each campaign and evaluate the results.

### Test Related Product Recommendations Campaign

Let's test the recommendations made by the related items/products campaign by selecting a product from the Retail Demo Store's [Products](https://github.com/aws-samples/retail-demo-store/tree/master/src/products) microservice and requesting related item recommendations for that product.

#### Select a Product

We'll just pick a random product for simplicity. Feel free to change the `product_id` below and execute the following cells with a different product to get a sense for how the recommendations change.

In [10]:
product_id = '020a5afe-fb13-4499-a1fa-8594d326eaa0'

#### Get Related Product Recommendations for Product

Now let's call Amazon Personalize to get related item/product recommendations for our product from the related item campaign.

In [11]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = related_campaign_arn,
    itemId = str(product_id),
    numResults = 10
)

item_list = get_recommendations_response['itemList']

In [12]:
print(json.dumps(item_list, indent=4))

[
    {
        "itemId": "eb8f10ab-1317-4a11-b058-b2098bb64326"
    },
    {
        "itemId": "a4c0f41d-4e7d-422c-86f1-57432c0fdba2"
    },
    {
        "itemId": "3f90e04e-9bfe-4fd4-a137-387b694baad2"
    },
    {
        "itemId": "ef446e39-c864-46ea-b273-f2a48b7dc2a5"
    },
    {
        "itemId": "57a7d4c1-03f7-4a5b-a618-cbfb5a0004f1"
    },
    {
        "itemId": "cacf945a-4c63-4797-a9a9-361e14b7001e"
    },
    {
        "itemId": "68e865bc-3db7-4f5d-86e3-8e7a651cf0b7"
    },
    {
        "itemId": "8bdfaf9c-4ff6-46bc-a304-33db265f36ef"
    },
    {
        "itemId": "2b8f89d0-4078-4701-8aac-89c48d8ba392"
    },
    {
        "itemId": "cacb5fe5-f77f-4bd8-979c-8eec17cb3255"
    }
]


Based on the random product selected above, do the similar item recommendations from Personalize make sense? Keep in mind that the similar item recommendations from the SIMS recipe are based on the interactions we generated as input into the solution creation process above.

### Test Product Recommendations Campaign

Let's test the recommendations made by the product recommendations campaign by selecting a user from the Retail Demo Store's Users microservice and requesting item recommendations for that user.

#### Select a User

We'll just pick a random user for simplicity. Feel free to change the `user_id` below and execute the following cells with a different user to get a sense for how the recommendations change.

In [13]:
user_id = 256

#### Get Product Recommendations for User

Now let's call Amazon Personalize to get recommendations for our user from the product recommendations campaign.

In [14]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = recommend_campaign_arn,
    userId = str(user_id),
    numResults = 10
)

item_list = get_recommendations_response['itemList']

In [15]:
print(json.dumps(item_list, indent=4))

[
    {
        "itemId": "f9456fda-e693-4046-a73f-64c9bedddad7",
        "score": 0.2523224
    },
    {
        "itemId": "2ed4b6f8-5d37-480e-9469-4ea26f475716",
        "score": 0.2005763
    },
    {
        "itemId": "523e5820-3024-4915-8d92-e191f68dee7f",
        "score": 0.0670596
    },
    {
        "itemId": "0ea1ce23-a279-407d-a68a-86c8b3e83f79",
        "score": 0.0553302
    },
    {
        "itemId": "133a1dbe-fbdd-4127-b456-f9ed790c5192",
        "score": 0.027842
    },
    {
        "itemId": "49b89871-5fe7-4898-b99d-953e15fb42b2",
        "score": 0.0267208
    },
    {
        "itemId": "9f274415-3a6c-4266-9de3-239a75ca36c1",
        "score": 0.0243126
    },
    {
        "itemId": "8a740ed4-b238-4131-b654-1e031c1ae7c6",
        "score": 0.0196197
    },
    {
        "itemId": "af4c855d-7897-41e3-b265-35941a64e659",
        "score": 0.0172376
    },
    {
        "itemId": "2a06c3e0-e349-496b-a549-da1b61a50b7a",
        "score": 0.0160845
    }
]


Notice that in this response we have a `score` field returned with each `itemId`. For all recipes except SIMS and Popularity-Count, Personalize [calculates a score](https://docs.aws.amazon.com/personalize/latest/dg/getting-real-time-recommendations.html) for each recommended item. Score values are between 0.0 and 1.0 and the sum of all scores across all items in your interactions and items datasets will total to 1.0. Therefore, the absolute value of scores will be smaller for larger item catalogs. We'll see how scores are calculated a bit differently for the personalized-ranking recipe below.


### Test Personalized Ranking Campaign

Next let's evaluate the results of the personalized ranking campaign. As a reminder, given a list of items and a user, this campaign will rerank the items based on the preferences of the user. For the Retail Demo Store, we will use this campaign to rerank the products listed for each category and the featured products list as well as reranking catalog search results displayed in the search widget.

#### Get Featured Products List

First let's get the list of featured products from the Products microservice.

In [16]:
# WE NEED A LIST OF FEATURED ITEMS HERE!!!!
# WE NEED A LIST OF FEATURED ITEMS HERE!!!!
# WE NEED A LIST OF FEATURED ITEMS HERE!!!!
# WE NEED A LIST OF FEATURED ITEMS HERE!!!!
# WE NEED A LIST OF FEATURED ITEMS HERE!!!!
# WE NEED A LIST OF FEATURED ITEMS HERE!!!!
# WE NEED A LIST OF FEATURED ITEMS HERE!!!!
# WE NEED A LIST OF FEATURED ITEMS HERE!!!!
# WE NEED A LIST OF FEATURED ITEMS HERE!!!!
# WE NEED A LIST OF FEATURED ITEMS HERE!!!!
# WE NEED A LIST OF FEATURED ITEMS HERE!!!!
# WE NEED A LIST OF FEATURED ITEMS HERE!!!!

#### ReRank Featured Products

Using the featured products list just retrieved, first we'll create a list of item IDs that we want to rerank for a specific user. This reranking will allow us to provide ranked products based on the user's behavior. These behaviors should be consistent the same persona that was mentioned above (since we're going to use the same `user_id`).

In [ ]:
unranked_product_ids = []

for product in featured_products:
    unranked_product_ids.append(product['id'])
    
print(', '.join(unranked_product_ids))

Now let's have Personalize rank the featured product IDs based on our random user.

In [ ]:
response = personalize_runtime.get_personalized_ranking(
    campaignArn=ranking_campaign_arn,
    inputList=unranked_product_ids,
    userId=str(user_id)
)
reranked = response['personalizedRanking']
print(json.dumps(response['personalizedRanking'], indent = 4))

Are the reranked results different than the original results from the Search service? Notice that we are also given a score for each item but this time the score values are larger. This is because scores for personalized-ranking results are calculated just across the items being reranked. Experiment with a different `user_id` in the cells above to see how the item ranking changes.

#### Pick products for discount

Using the featured products list we'll pick some products for discount from the featured products.

We'll get the ranking when discount context is applied for comparison. This is a using the "contextual metadata" feature of Amazon Personalize.


In [ ]:
response = personalize_runtime.get_personalized_ranking(
    campaignArn=ranking_campaign_arn,
    inputList=unranked_product_ids,
    userId=str(user_id),
    context={'DISCOUNT': 'Yes'} # Here we provide the context for the ranking
)
disount_reranked = response['personalizedRanking']
print('Discount context ranking:', json.dumps(disount_reranked, indent = 4))
print('Discount:', [item['itemId'] for item in disount_reranked[:2]])

We could use the discount-context ranking directly, but what we might be more interested in seeing is those products that
benefit from having a discout shown. In our simulated data, certain products are more likely to see 
purchases with discount (to be precise, the cheaper ones). Let us find out which products benefit most. We also make use of the scores returned by Personalize when it returns the ranking.

In [ ]:
eps = 0.00001 #  "epsilon" - a number slightly more than zero so we don't get division by zero
non_discount_rerank_scores = {item['itemId']: max(item['score'], eps) for item in reranked}
discount_rerank_scores = {item['itemId']: item['score'] for item in disount_reranked}
score_increases_with_discount = {item_id: discount_rerank_scores[item_id]/non_discount_rerank_scores[item_id]
                                 for item_id in discount_rerank_scores}
# Let us get the sorted items:
discount_improve_sorted_items = sorted(score_increases_with_discount.keys(),
                                       key=lambda key: score_increases_with_discount[key])

print('Improvement ranking:', discount_improve_sorted_items)
# Let us pick the two items that respond best to discounts
print('Discount:', discount_improve_sorted_items[:2])

Has the ranking changed?

## Event Tracking - Keeping up with evolving user intent

Up to this point we have trained and deployed three Amazon Personalize campaigns based on historical data that we
 generated in this workshop. This allows us to make related product, user recommendations, and rerank product
  lists based on already observed behavior of our users. However, user intent often changes in real-time such
  that what products the user is interested in now may be different than what they were interested in a week ago,
  a day ago, or even a few minutes ago. Making recommendations that keep up with evolving user intent is one of the
   more difficult challenges with personalization. Fortunately, Amazon Personalize has a mechanism for this exact issue.

Amazon Personalize supports the ability to send real-time user events (i.e. clickstream) data into the service.
Personalize uses this event data to improve recommendations. It will also save these events and automatically
include them when solutions for the same dataset group are re-created (i.e. model retraining).

The Retail Demo Store's Web UI already has
[logic to send events](https://github.com/aws-samples/retail-demo-store/blob/master/src/web-ui/src/analytics/AnalyticsHandler.js)
such as 'ProductViewed', 'ProductAdded', 'OrderCompleted', and others as they occur in real-time to a Personalize Event Tracker.
These are the same event types we used to initially create the solutions and campaigns for our three use-cases.
All we need to do is create an event tracker in Personalize, set the tracking Id for the tracker in an SSM parameter,
and rebuild the Web UI service to pick up the change.

### Create Personalize Event Tracker

Let's start by creating an event tracker for our dataset group.

In [18]:
event_tracker_response = personalize.create_event_tracker(
    datasetGroupArn=dataset_group_arn,
    name='retaildemostore-event-tracker'
)

event_tracker_arn = event_tracker_response['eventTrackerArn']
event_tracking_id = event_tracker_response['trackingId']

print('Event Tracker ARN: ' + event_tracker_arn)
print('Event Tracking ID: ' + event_tracking_id)

Event Tracker ARN: arn:aws:personalize:us-east-1:144386903708:event-tracker/63e3c48f
Event Tracking ID: b656f112-2258-4547-92dd-1c29f68ec966


### Wait for Event Tracker Status to Become ACTIVE

The event tracker should take a minute or so to become active.

In [19]:
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_event_tracker_response = personalize.describe_event_tracker(
        eventTrackerArn = event_tracker_arn
    )
    status = describe_event_tracker_response["eventTracker"]["status"]
    print("EventTracker: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

EventTracker: CREATE PENDING
EventTracker: ACTIVE


### Cold User Recommendations

One of the key features of Personalize is being able to cold start users. Cold users are typically those who are new to your site or application and cold starting a user is getting from no personalization to making personalized recommendations in real-time. 

Personalize accomplishes cold starting users via the Event Tracker, just as we saw above with existing users. However, since new users are typically anonymous for a period of time before they create an account or may choose to transact as a guest, personalization is a valuable tool to help convert those anonymous users to transacting users. 

The challenge here is that Personalize needs a `userId` for anonymous users before it can make personalized recommendations. The Retail Demo Store solves this challenge by creating a provisional user ID the moment an anonymous user first hits the site. This provisional user ID is then used when streaming events to the Event Tracker and when retrieving recommendations from the Recommendations service. This allows the Retail Demo Store to start serving personalized recommendations after the first couple events are streamed to Personalize. Before recommendations can be personalized, Personalize will provide recommendations for popular items as a fallback.

To see this behavior in action, browse to the Retail Demo Store storefront using a different browser, an Incognito/Private window, or sign out of your existing account. What you should see on the home page is that instead of **"Inspired by your shopping behavior"**, the section is **"Trending products"**. After you click on a couple provide detail pages, return to the home page and see that the section title and recommendations have changed. This indicates that recommendations are now being personalized and will continue to become more relevant as you engage with products.

Similarly, the category pages will rerank products at first based on popularity and then become more and more personalized.

There are some challenges with this approach, though. First is the question of what to do with the provisional user ID when the user creates an account. To maintain continuity of the user's interaction history, the Retail Demo Store passes the provisional user ID to the Users microservice when creating a new user account. The Users service then uses this ID as the user's ID going forward. Another challenge is how to handle a user that anonymously browses the site using multiple devices such as on the mobile device and then on a desktop/laptop. In this case, separate provisional user IDs are generated for sessions on each device. However, once the user creates an account on one device and then signs in with that account on the other device, both devices will starting using the same user ID going forward. A side effect here is that the interaction history from one of the devices will be orphaned. This is an acceptable tradeoff given the benefit of cold starting users earlier and is functionally the same UX without this scheme. Additional logic could be added to merge the interaction history from both prior anonymous sessions when the user creates an account. Also, customer data platforms can be used to help manage this for you.

### Test Purchased Products Filter

To test our purchased products filter, we will request recommendations for a random user. Then we will send an `OrderCompleted` event for one of the recommended products to Personalize using the event tracker created above. Finally, we will request recommendations again for the same user but this time specify our filter.

In [20]:
# Pick a user ID in the range of test users and fetch 5 recommendations.
user_id = '456'
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = recommend_campaign_arn,
    userId = user_id,
    numResults = 5
)

item_list = get_recommendations_response['itemList']
print(json.dumps(item_list, indent=2))

[
  {
    "itemId": "dfd7c361-dc70-4bb4-9c05-e6357ecabc49",
    "score": 0.1654873
  },
  {
    "itemId": "323ca3fe-7849-490a-933d-e742866a2843",
    "score": 0.1571613
  },
  {
    "itemId": "5b53ab8d-701c-4139-bdab-dc457e546157",
    "score": 0.138697
  },
  {
    "itemId": "b2dba8a2-1634-4579-8b46-b52632f49de2",
    "score": 0.0311615
  },
  {
    "itemId": "0c4744e2-b989-4509-a7e2-7d8dc43ff404",
    "score": 0.0261859
  }
]


Next let's randomly select an item from the returned list of recommendations to be our product to purchase.

In [21]:
product_id_to_purchase = random.choice(item_list)['itemId']
print(f'Product to simulate purchasing: {product_id_to_purchase}')

Product to simulate purchasing: b2dba8a2-1634-4579-8b46-b52632f49de2


Next let's send an `OrderCompleted` event to Personalize to simulate that the product was just purchased.
This will match the criteria for our filter.
In the Retail Demo Store web application, this event is sent for each product in the order after the order is completed.

In [22]:
response = personalize_events.put_events(
    trackingId = event_tracking_id,
    userId = user_id,
    sessionId = str(uuid.uuid4()),
    eventList = [
        {
            'eventId': str(uuid.uuid4()),
            'eventType': 'OrderCompleted',
            'itemId': str(product_id_to_purchase),
            'sentAt': int(time.time()),
            'properties': '{"discount": "No"}'
        }
    ]
)

# Wait for OrderCompleted event to become consistent.
time.sleep(5)

print(json.dumps(response, indent=2))

{
  "ResponseMetadata": {
    "RequestId": "51a298db-a8dc-4266-b754-171d7fe5a793",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/json",
      "date": "Thu, 18 Feb 2021 20:21:47 GMT",
      "x-amzn-requestid": "51a298db-a8dc-4266-b754-171d7fe5a793",
      "content-length": "0",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Finally, let's retrieve recommendations for the user again but this time specifying the filter to exclude recently
purchased items. We do this by passing the filter's ARN via the `filterArn` parameter.
In the Retail Demo Store, this is done in the
[Recommendations](https://github.com/aws-samples/retail-demo-store/tree/master/src/recommendations) service.

In [23]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = recommend_campaign_arn,
    userId = user_id,
    numResults = 5,
    filterArn = filter_arn
)

item_list = get_recommendations_response['itemList']
print(json.dumps(item_list, indent=2))

[
  {
    "itemId": "323ca3fe-7849-490a-933d-e742866a2843",
    "score": 0.1993862
  },
  {
    "itemId": "dfd7c361-dc70-4bb4-9c05-e6357ecabc49",
    "score": 0.1550834
  },
  {
    "itemId": "5b53ab8d-701c-4139-bdab-dc457e546157",
    "score": 0.1326684
  },
  {
    "itemId": "59bb3cc4-9757-49a0-ac62-8c0afe105a3d",
    "score": 0.0345377
  },
  {
    "itemId": "0c4744e2-b989-4509-a7e2-7d8dc43ff404",
    "score": 0.0246912
  }
]


The following code will raise an assertion error if the product we just purchased is still recommended.

In [24]:
found_item = next((item for item in item_list if item['itemId'] == product_id_to_purchase), None)
if found_item:
    assert found_item == False, 'Purchased item found unexpectedly in recommendations'
else:
    print('Purchased item filtered from recommendations for user!')

Purchased item filtered from recommendations for user!


## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, cut a pull request and provide any additional helpful components that may be missing from this collection.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in `05_Clean_Up_Resources.ipynb`.

In [25]:
%store event_tracker_arn

Stored 'event_tracker_arn' (str)
